In [136]:
# First ensemble with NSL-KDD
# Parameters

#----------------------------------------------
# 0 for not using it as base learner
# 1 for using it as base learner

use_model_ada = 1 
use_model_dnn = 1 
use_model_mlp = 1 
use_model_lgbm = 1 
use_model_rf = 1 
use_model_svm = 1
use_model_knn = 1 
#----------------------------------------------
# 0 for training the model
# 1 for using the saved version of the model

load_model_ada = 0 
load_model_dnn = 0 
load_model_mlp = 0 
load_model_lgbm = 0 
load_model_rf = 0 
load_model_svm = 0
load_model_knn = 0 
#----------------------------------------------





In [137]:

# Specify the name of the output text file
output_file_name = "My_fst_ensemble.txt"
with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
with open(output_file_name, "a") as f: print('---- Start Ensemble Model Info - v0 ----', file = f)


In [138]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
# importing required libraries
import numpy as np
import pandas as pd
import pickle # saving and loading trained model
from os import path


# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense # importing dense layer

from keras.layers import Input
from keras.models import Model
# representation of model layers
#from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import shap




# In[2]:


#Defining metric functions
def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    eps = 1e-7
    Precision = TP/(TP+FP+eps)
    

    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    eps = 1e-7
    MCC = (TN*TP-FN*FP)/(((TP+FP+eps)*(TP+FN+eps)*(TN+FP+eps)*(TN+FN+eps))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
     # plt.plot(fpr[i], tpr[i], color='darkorange', lw=2)
      #print('AUC for Class {}: {}'.format(i+1, auc(fpr[i], tpr[i])))
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting


# In[3]:


# attach the column names to the dataset
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.

train='KDDTrain+.txt'
test='KDDTest+.txt'

df=pd.read_csv(train,names=feature)
df_test=pd.read_csv(test,names=feature)

In [139]:


# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)


df.drop(['difficulty'],axis=1,inplace=True)
df_test.drop(['difficulty'],axis=1,inplace=True)



print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())



# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())



# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()


# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2




df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)



enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()


trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape

newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)


# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())


# Specify your selected features. Note that you'll need to modify this list according to your final processed dataframe
#Uncomment the below lines to use these top 20 features from shap analysis
#selected_features = ["root_shell","service_telnet","num_shells","service_uucp","dst_host_same_src_port_rate"
#                     ,"dst_host_rerror_rate","dst_host_srv_serror_rate","dst_host_srv_count","service_private","logged_in",
#                    "dst_host_serror_rate","serror_rate","srv_serror_rate","flag_S0","diff_srv_rate","dst_host_srv_diff_host_rate","num_file_creations","flag_RSTR"#,"dst_host_same_srv_rate","service_Idap","label"]
                     

# Select those features from your dataframe
#newdf = newdf[selected_features]
#newdf_test = newdf_test[selected_features]

# Now your dataframe only contains your selected features.

# creating a dataframe with multi-class labels (Dos,Probe,R2L,U2R,normal)
multi_data = newdf.copy()
multi_label = pd.DataFrame(multi_data.label)

multi_data_test=newdf_test.copy()
multi_label_test = pd.DataFrame(multi_data_test.label)


# using standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = multi_data.select_dtypes(include='number').columns
data = standardization(multi_data,numeric_col)
numeric_col_test = multi_data_test.select_dtypes(include='number').columns
data_test = standardization(multi_data_test,numeric_col_test)

# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
le2_test = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
enc_label_test = multi_label_test.apply(le2_test.fit_transform)
multi_data = multi_data.copy()
multi_data_test = multi_data_test.copy()

multi_data['intrusion'] = enc_label
multi_data_test['intrusion'] = enc_label_test

#y_mul = multi_data['intrusion']
multi_data
multi_data_test



multi_data.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data
multi_data_test.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data_test


y_train_multi= multi_data[['intrusion']]
X_train_multi= multi_data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:',X_train_multi.shape,'\ny_train has shape:',y_train_multi.shape)

y_test_multi= multi_data_test[['intrusion']]
X_test_multi= multi_data_test.drop(labels=['intrusion'], axis=1)

print('X_test has shape:',X_test_multi.shape,'\ny_test has shape:',y_test_multi.shape)


from collections import Counter

label_counts = Counter(y_train_multi['intrusion'])
print(label_counts)


from sklearn.preprocessing import LabelBinarizer

y_train_multi = LabelBinarizer().fit_transform(y_train_multi)

y_test_multi = LabelBinarizer().fit_transform(y_test_multi)


Y_train=y_train_multi.copy()
X_train=X_train_multi.copy()

Y_test=y_test_multi.copy()
X_test=X_test_multi.copy()




Dimensions of the Training set: (125973, 43)
Dimensions of the Test set: (22544, 43)
Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

Label distribution Test set:
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess  

(125973, 123)
(22544, 123)
0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64
X_train has shape: (125973, 122) 
y_train has shape: (125973, 1)
X_test has shape: (22544, 122) 
y_test has shape: (22544, 1)
Counter({0: 67343, 1: 45927, 2: 11656, 3: 995, 4: 52})


In [140]:

# In[24]:

'''
from sklearn.feature_selection import SelectKBest, f_classif

# Number of best features you want to select
k = 15

# Initialize a dataframe to store the scores for each feature against each class
feature_scores = pd.DataFrame(index=X_train.columns)

# Loop through each class
for class_index in range(Y_train.shape[1]):
    
    # Get the current class labels
    y_train_current_class = Y_train[:, class_index]
    
    # Select K best features for the current class
    best_features = SelectKBest(score_func=f_classif, k='all')
    fit = best_features.fit(X_train, y_train_current_class)

    # Get the scores
    df_scores = pd.DataFrame(fit.scores_, index=X_train.columns, columns=[f"class_{class_index}"])
    
    # Concatenate the scores to the main dataframe
    feature_scores = pd.concat([feature_scores, df_scores],axis=1)

# Get the sum of the scores for each feature
feature_scores['total'] = feature_scores.sum(axis=1)

# Get the top k features in a list
top_k_features = feature_scores.nlargest(k, 'total').index.tolist()

print(top_k_features)

'''
# In[32]:

'\nfrom sklearn.feature_selection import SelectKBest, f_classif\n\n# Number of best features you want to select\nk = 15\n\n# Initialize a dataframe to store the scores for each feature against each class\nfeature_scores = pd.DataFrame(index=X_train.columns)\n\n# Loop through each class\nfor class_index in range(Y_train.shape[1]):\n    \n    # Get the current class labels\n    y_train_current_class = Y_train[:, class_index]\n    \n    # Select K best features for the current class\n    best_features = SelectKBest(score_func=f_classif, k=\'all\')\n    fit = best_features.fit(X_train, y_train_current_class)\n\n    # Get the scores\n    df_scores = pd.DataFrame(fit.scores_, index=X_train.columns, columns=[f"class_{class_index}"])\n    \n    # Concatenate the scores to the main dataframe\n    feature_scores = pd.concat([feature_scores, df_scores],axis=1)\n\n# Get the sum of the scores for each feature\nfeature_scores[\'total\'] = feature_scores.sum(axis=1)\n\n# Get the top k features in a l

In [141]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification

# Assuming you have features X and labels Y
# X, Y = make_classification()

ros = RandomOverSampler(sampling_strategy='minority', random_state=100)

X_train, Y_train = ros.fit_resample(X_train, Y_train)


# In[33]:


print(Y_test)


# In[34]:


X_train.values


# In[35]:



[[0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 ...
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]


array([[-1.10249223e-01, -7.67859947e-03, -4.91864438e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-1.10249223e-01, -7.73736981e-03, -4.91864438e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-1.10249223e-01, -7.76224074e-03, -4.91864438e-03, ...,
        -1.97262160e-02, -1.21190076e+00, -4.64315895e-02],
       ...,
       [-9.29714678e-02, -7.36430591e-03, -3.87394518e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-8.68282658e-02, -7.36430591e-03, -3.87568593e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [ 1.61587463e-01, -7.46804833e-03,  1.06953862e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02]])

In [142]:
single_class_train = np.argmax(y_train_multi, axis=1)
single_class_test = np.argmax(y_test_multi, axis=1)


df1 = X_train_multi.assign(Label = single_class_train)
df2 =  X_test_multi.assign(Label = single_class_test)

frames = [df1,  df2]

df = pd.concat(frames,ignore_index=True)

y = df.pop('Label')
X = df

y1, y2 = pd.factorize(y)

y_0 = pd.DataFrame(y1)
y_1 = pd.DataFrame(y1)
y_2 = pd.DataFrame(y1)
y_3 = pd.DataFrame(y1)
y_4 = pd.DataFrame(y1)


# y_0 = y_0.replace(0, 0)
# y_0 = y_0.replace(1, 1)
y_0 = y_0.replace(2, 1)
y_0 = y_0.replace(3, 1)
y_0 = y_0.replace(4, 1)


y_1 = y_1.replace(1, 999)
y_1 = y_1.replace(0, 1)
# y_1 = y_1.replace(1, 0)
y_1 = y_1.replace(2, 1)
y_1 = y_1.replace(3, 1)
y_1 = y_1.replace(4, 1)
y_1 = y_1.replace(999, 1)


y_2 = y_2.replace(0, 1)
y_2 = y_2.replace(1, 1)
y_2 = y_2.replace(2, 0)
y_2 = y_2.replace(3, 1)
y_2 = y_2.replace(4, 1)


y_3 = y_3.replace(0, 1)
# y_3 = y_3.replace(1, 1)
y_3 = y_3.replace(2, 1)
y_3 = y_3.replace(3, 0)
y_3 = y_3.replace(4, 1)


y_4 = y_4.replace(0, 1)
# y_4 = y_4.replace(1, 1)
y_4 = y_4.replace(2, 1)
y_4 = y_4.replace(3, 1)
y_4 = y_4.replace(4, 0)



df = df.assign(Label = y)

In [143]:
#Divide the dataset between level 00 and level 01
import sklearn
from sklearn.model_selection import train_test_split
split = 0.6 # 0.7

# X_00,X_01, y_00, y_01 = sklearn.model_selection.train_test_split(X, y, train_size=split)
X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=split)

In [144]:
from collections import Counter

label_counts2 = Counter(y)
print(label_counts2)


Counter({0: 77054, 1: 53387, 2: 14077, 3: 3880, 4: 119})


In [145]:
#Base learner Split
# split = 0.7

# X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_00, y_00, train_size=split)

In [146]:
X_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
4157,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
89149,-0.109865,-0.007463,-0.004828,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
44682,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
97916,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
105826,-0.052657,-0.007733,0.030866,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,0.030104,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42108,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
39197,-0.110249,-0.007707,-0.004289,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
35922,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
11744,-0.110249,-0.007714,-0.004659,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432


In [147]:
y_train

4157      1
89149     0
44682     1
97916     1
105826    0
         ..
42108     1
39197     0
35922     1
11744     0
132542    1
Name: Label, Length: 89110, dtype: int64

## LEVEL 0 - Weak models - Base Learner

In [148]:
with open(output_file_name, "a") as f: print('------------START of WEAK LEARNERS (BASE MODELS) - STACK 00 -----------------', file = f)

#Defining Basemodels


print('---------------------------------------------------------------------------------')
print('Defining RF Model')
print('---------------------------------------------------------------------------------')
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
#------------------------------------------------------------------------------


print('---------------------------------------------------------------------------------')
print('Defining ADA Model')
print('---------------------------------------------------------------------------------')
#ADA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
import time
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)


print('---------------------------------------------------------------------------------')
print('Defining LGBM Model')
print('---------------------------------------------------------------------------------')
#LGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()



#KNN
print('---------------------------------------------------------------------------------')
print('Defining KNN Model')
print('---------------------------------------------------------------------------------')
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors = 5)


#SVM
print('---------------------------------------------------------------------------------')
print('Defining SVM Model')
print('---------------------------------------------------------------------------------')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)


#MLP
print('---------------------------------------------------------------------------------')
print('Defining MLP Model')
print('---------------------------------------------------------------------------------')


from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import time

# create MLPClassifier instance
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)


#DNN
print('---------------------------------------------------------------------------------')
print('Defining DNN Model')
print('---------------------------------------------------------------------------------')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Model Parameters
dropout_rate = 0.01
nodes = 70
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=1
batch_size=2*256


num_columns = X_train.shape[1]

dnn = tf.keras.Sequential()

# Input layer
dnn.add(tf.keras.Input(shape=(num_columns,)))

# Dense layers with dropout
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

# Output layer
dnn.add(tf.keras.layers.Dense(out_layer))



dnn.compile(optimizer=optimizer, loss=loss)

dnn.summary()



# dnn = Sequential()
# dnn.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer
# dnn.add(Dense(64, activation='relu'))  # Hidden layer
# dnn.add(Dense(5))  # Output layer

# dnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # summary of model layers
# dnn.summary()

---------------------------------------------------------------------------------
Defining RF Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining ADA Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining LGBM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining KNN Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining SVM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Def

In [149]:
# #SVM
# # Wrap SGDClassifier with MultiOutputClassifier
# multi_target_clf = MultiOutputClassifier(clf)

# # Fit the model on the training data
# multi_target_clf.fit(X_train, y_train)

# Make predictions on the test data
# y_pred = clf.predict(X_test)



In [150]:
#Training Basemodels
import joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score
n_splits = 5  # You can adjust the number of folds as needed



print('---------------------------------------------------------------------------------')
print('Training Model')
with open(output_file_name, "a") as f: print('Training weak models - level 0', file = f)

print('---------------------------------------------------------------------------------')

if use_model_ada == 1 and load_model_ada == 0:

    print('---------------------------------------------------------------------------------')
    print('Training ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA


    start = time.time()
    ada = abc.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(ada, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)

    # Assuming 'model' is your trained model
    joblib.dump(ada, 'ada_base_model.joblib')


if use_model_rf == 1 and load_model_rf == 0:

    print('---------------------------------------------------------------------------------')
    print('Training RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('Training RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    model_rf = rf.fit(X_train,y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(model_rf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(model_rf, 'rf_base_model.joblib')

if use_model_svm == 1 and load_model_svm == 0:

    print('---------------------------------------------------------------------------------')
    print('Training SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM

    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    # clf.score(X_train, y_train)
    time_taken = end - start

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(clf, 'svm_base_model.joblib')


if use_model_knn == 1 and load_model_knn == 0:

    #KNN
    print('---------------------------------------------------------------------------------')
    print('Training KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training KNN', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    knn_clf.fit(X_train,y_train)
    end = time.time()


    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(knn_clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(knn_clf, 'knn_base_model.joblib')


if use_model_lgbm == 1 and load_model_lgbm == 0:


    print('---------------------------------------------------------------------------------')
    print('Training LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    lgbm.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(lgbm, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(lgbm, 'lgbm_base_model.joblib')

if use_model_mlp == 1 and load_model_mlp == 0:


    print('---------------------------------------------------------------------------------')
    print('Training MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training MLP', file = f)
    print('---------------------------------------------------------------------------------')

    start = time.time()
    MLP = mlp.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(MLP, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(MLP, 'mlp_base_model.joblib')


if use_model_dnn == 1 and load_model_dnn == 0:

    print('---------------------------------------------------------------------------------')
    print('Training DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training DNN', file = f)
    print('---------------------------------------------------------------------------------')
    # Convert Y_test back to its original format
    # y_test = np.argmax(Y_test, axis=1)

    # Start the timer
    start = time.time()
    dnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    # End the timer
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(dnn, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    dnn.save("DNN_base_model.h5")

    # Calculate the time taken and print it out
    # print(f'Time taken for training: {time_taken} seconds')


with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)



---------------------------------------------------------------------------------
Training Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training ADA
---------------------------------------------------------------------------------


Cross-validation scores: [0.80888789 0.82751655 0.82269106 0.59852991 0.73291438]
Mean accuracy: 0.75810795645831
---------------------------------------------------------------------------------
Training RF
---------------------------------------------------------------------------------
Cross-validation scores: [0.94529234 0.93956907 0.94226237 0.94428235 0.9381102 ]
Mean accuracy: 0.9419032656267534
---------------------------------------------------------------------------------
Training SVM
---------------------------------------------------------------------------------
Cross-validation scores: [0.96835372 0.96919538 0.96812928 0.96627763 0.96711929]
Mean accuracy: 0.9678150600381551
---------------------------------------------------------------------------------
Training KNN
---------------------------------------------------------------------------------
Cross-validation scores: [0.98905847 0.98776793 0.98877791 0.98984401 0.98990012]
Mean accuracy: 0.9890696891482437
--------

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x7f9762adf358>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [151]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Define your Keras model as a function
def create_model(optimizer='adam', hidden_layer_size=16):
    # model = Sequential()
    # model.add(Dense(hidden_layer_size, input_dim=input_size, activation='relu'))
    # model.add(Dense(1, activation='sigmoid'))
    # model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        
    dnn = tf.keras.Sequential()

    # Input layer
    dnn.add(tf.keras.Input(shape=(num_columns,)))

    # Dense layers with dropout
    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    dnn.add(tf.keras.layers.Dense(out_layer))



    dnn.compile(optimizer=optimizer, loss=loss)

    dnn.summary()
    return dnn

# Create a KerasClassifier
dnn = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the parameter grid for GridSearchCV
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'hidden_layer_size': [8, 16, 32]
}

# Create the StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create GridSearchCV
grid = GridSearchCV(estimator=dnn, param_grid=param_grid, cv=cv, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and best accuracy
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 70)                8610      
_________________________________________________________________
dropout_40 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_41 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_42 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 70)               

In [ ]:
# stratified_kfold

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [ ]:
# Loading Models
from tensorflow.keras.models import load_model

if load_model_ada == 1:
    ada = loaded_model = joblib.load('ada_base_model.joblib')

if load_model_svm == 1:
    clf = loaded_model = joblib.load('svm_base_model.joblib')

if load_model_dnn == 1:
    dnn = load_model("DNN_base_model.h5")

if load_model_knn == 1:
    knn_clf = loaded_model = joblib.load('knn_base_model.joblib')

if load_model_mlp == 1:
    MLP = loaded_model = joblib.load('mlp_base_model.joblib')

if load_model_rf == 1:
    model_rf = loaded_model = joblib.load('rf_base_model.joblib')

if load_model_lgbm == 1:
    lgbm = loaded_model = joblib.load('lgbm_base_model.joblib')







In [ ]:
# Make predictions on the test data
# preds_svm = clf.predict(X_test)



# y_scores = y_pred
# y_true = y_test



### Base leaners predictions

In [ ]:
with open(output_file_name, "a") as f: print('Generating Predictions', file = f)

if use_model_rf == 1:

    print('---------------------------------------------------------------------------------')
    print('Prediction RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    preds_rf = rf.predict(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_svm == 1:

    print('Prediction SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM
    start = time.time()
    preds_svm = clf.predict(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_lgbm == 1:

    print('Prediction LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    #LGBM
    start = time.time()
    preds_lgbm = lgbm.predict(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_dnn == 1:

    print('Prediction DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction DNN', file = f)
    print('---------------------------------------------------------------------------------')
    #DNN
    start = time.time()
    pred_dnn = dnn.predict(X_test)
    preds_dnn = np.argmax(pred_dnn,axis = 1)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_ada == 1:

    print('Prediction ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA
    start = time.time()
    preds_ada = ada.predict(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction MLP', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_mlp == 1:

    #MLP
    start = time.time()
    y_pred = MLP.predict_proba(X_test)
    preds_mlp = np.argmax(y_pred,axis = 1)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction KNN', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_knn == 1:

    #KNN
    start = time.time()
    preds_knn =knn_clf.predict(X_test)
    preds_knn
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)


---------------------------------------------------------------------------------
Prediction RF
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction SVM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction LGBM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction DNN
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction ADA
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction MLP
--------

### METRICS - Base Learners

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score



# >>> 
# >>> roc_auc_score(y, clf.predict_proba(X)[:, 1])
# 0.99...
# >>> roc_auc_score(y, clf.decision_function(X))

#### RF

In [ ]:
# y_test
# pred_label

In [ ]:
#RF
if use_model_rf == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('RF base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_rf
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  30589.0    125.0    50.0    0.0  0.0
1   1168.0  20127.0    14.0    0.0  0.0
2    868.0     71.0  4768.0    0.0  0.0
3   1209.0     47.0    19.0  308.0  0.0
4     44.0      0.0     0.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9391485851835643
Precision total:  0.774775432966692
Recall total:  0.5937749017028259
F1 total:  0.6282355205511393
BACC total:  0.5937749017028259
MCC total:  0.8974246859100513


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#DNN
if use_model_dnn == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('DNN base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_dnn
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3       4
0  26866.0   2849.0   786.0  130.0   133.0
1   1107.0  16468.0  1951.0  344.0  1439.0
2   3182.0    380.0  2014.0   25.0   106.0
3    279.0   1012.0   258.0   21.0    13.0
4      6.0     12.0    19.0    1.0     6.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.7637988789199925
Precision total:  0.4187330923884616
Recall total:  0.42972839821586745
F1 total:  0.40990190348146227
BACC total:  0.42972839821586745
MCC total:  0.6020474809587878


In [ ]:
#ADA
if use_model_ada == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('ADA base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_ada
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3     4
0  29270.0    944.0   351.0  173.0  26.0
1   1512.0  15605.0  4185.0    7.0   0.0
2   2155.0    297.0  3247.0    8.0   0.0
3    690.0      0.0    11.0  882.0   0.0
4     17.0      0.0     1.0    5.0  21.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8252394498964768
Precision total:  0.6960294069942633
Recall total:  0.6574298981875355
F1 total:  0.6666067458843546
BACC total:  0.6574298981875355
MCC total:  0.710375854100207


In [ ]:
#SVM
if use_model_svm == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('SVM base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_svm
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3     4
0  30314.0    178.0   183.0   87.0   2.0
1    409.0  20862.0    37.0    1.0   0.0
2    252.0     85.0  5354.0   16.0   0.0
3    596.0      2.0     8.0  977.0   0.0
4     25.0      0.0     0.0    6.0  13.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9682360664568148
Precision total:  0.9343014383933239
Recall total:  0.7630357415812434
F1 total:  0.8152989443588116
BACC total:  0.7630357415812434
MCC total:  0.9462063278330396


In [ ]:
#KNN
if use_model_knn == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('KNN base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_knn
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2       3     4
0  30590.0     27.0    51.0    94.0   2.0
1     41.0  21267.0     1.0     0.0   0.0
2    101.0    141.0  5460.0     5.0   0.0
3     98.0      7.0     2.0  1474.0   2.0
4     13.0      0.0     0.0     3.0  28.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9901021765111856
Precision total:  0.9568243096961192
Recall total:  0.9033199786116558
F1 total:  0.9262434354690277
BACC total:  0.9033199786116558
MCC total:  0.9832985858386466


In [ ]:
#MLP
if use_model_mlp == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('MLP base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_mlp
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2       3     4
0  30579.0     23.0    32.0   125.0   5.0
1     28.0  21275.0     6.0     0.0   0.0
2    130.0     20.0  5548.0     9.0   0.0
3     45.0      0.0     0.0  1537.0   1.0
4     15.0      0.0     0.0     6.0  23.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9925093002508122
Precision total:  0.9387445263596114
Recall total:  0.891639781821078
F1 total:  0.9094569235433629
BACC total:  0.891639781821078
MCC total:  0.9873749014621037


In [ ]:
#lgbm

if use_model_lgbm == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('LGBM base model', file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    pred_label = preds_lgbm
    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')

    # Acc = ACC(TP_total,TN_total, FP_total, FN_total)
    # Precision = PRECISION(TP_total, FP_total)
    # Recall = RECALL(TP_total, FN_total)
    # F1 = F1(Recall,Precision)
    # BACC = BACC(TP_total,TN_total, FP_total, FN_total)
    # MCC = MCC(TP_total,TN_total, FP_total, FN_total)

    Acc = accuracy_score(y_test, pred_label)
    Precision = precision_score(y_test, pred_label, average='macro')
    Recall = recall_score(y_test, pred_label, average='macro')
    F1 =  f1_score(y_test, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test, pred_label)
    MCC = matthews_corrcoef(y_test, pred_label)

    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2       3     4
0  30418.0    128.0    83.0    82.0  53.0
1    343.0  20863.0    85.0    17.0   1.0
2     85.0     31.0  5563.0    28.0   0.0
3     81.0      3.0     0.0  1492.0   7.0
4     24.0      2.0     0.0     9.0   9.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9821233188008147
Precision total:  0.7981397650068244
Recall total:  0.8179300923297955
F1 total:  0.8062542642352568
BACC total:  0.8179300923297955
MCC total:  0.969893387766871


## Training the stronger model - STACK level 01

In [ ]:
with open(output_file_name, "a") as f: print('------------START of STRONGER LEARNER - STACK 01 -----------------', file = f)


# Stack the vectors horizontally to create a matrix
column_features = ['dnn','rf','lgbm','ada','knn','mlp','svm','label']
training_matrix = np.column_stack((
                          preds_dnn,
                          preds_rf,
                          preds_lgbm,
                          preds_ada,
                          preds_knn, 
                          preds_mlp,
                          preds_svm,
                          y_test
                          ))

# Print the resulting matrix
print(training_matrix)

[[0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
df_level_01 = pd.DataFrame(training_matrix, columns=column_features)

In [ ]:
y_01 = df_level_01.pop('label')
X_01 = df_level_01
df_level_01 = df_level_01.assign(label = y_01)

In [ ]:
X_01

,dnn,rf,lgbm,ada,knn,mlp,svm
0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
59402,1,1,1,1,1,1,1
59403,0,0,0,0,0,0,0
59404,0,0,0,0,0,0,0
59405,0,0,0,0,0,0,0


In [ ]:
y_01

0        0
1        1
2        0
3        0
4        0
        ..
59402    1
59403    0
59404    0
59405    0
59406    0
Name: label, Length: 59407, dtype: int64

In [ ]:
df_level_01

,dnn,rf,lgbm,ada,knn,mlp,svm,label
0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
59402,1,1,1,1,1,1,1,1
59403,0,0,0,0,0,0,0,0
59404,0,0,0,0,0,0,0,0
59405,0,0,0,0,0,0,0,0


In [ ]:
split = 0.7

X_train_01,X_test_01, y_train_01, y_test_01 = sklearn.model_selection.train_test_split(X_01, y_01, train_size=split)

In [ ]:
#DNN
print('---------------------------------------------------------------------------------')
print('Defining DNN Model')
print('---------------------------------------------------------------------------------')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Model Parameters
dropout_rate = 0.01
nodes = 70
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=5
batch_size=2*256


num_columns = X_train_01.shape[1]

dnn_01 = tf.keras.Sequential()

# Input layer
dnn_01.add(tf.keras.Input(shape=(num_columns,)))

# Dense layers with dropout
dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn_01.add(tf.keras.layers.Dense(nodes))
dnn_01.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

# Output layer
dnn_01.add(tf.keras.layers.Dense(out_layer))



dnn_01.compile(optimizer=optimizer, loss=loss)

dnn_01.summary()

---------------------------------------------------------------------------------
Defining DNN Model
---------------------------------------------------------------------------------
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 70)                560       
_________________________________________________________________
dropout_30 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 70)                4970      
_________________________________________________________________
dropout_31 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 70)                4970      
_________________________________________________________________
dro

In [ ]:
print('---------------------------------------------------------------------------------')
print('Training DNN')
with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('Training DNN', file = f)
print('---------------------------------------------------------------------------------')
# Convert Y_test back to its original format
# y_test = np.argmax(Y_test, axis=1)

# Start the timer
start = time.time()
dnn_01.fit(X_train_01, y_train_01, epochs=epochs, batch_size=batch_size)
# End the timer
end = time.time()
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
# joblib.dump(dnn_01, 'dnn_level_01.joblib')
dnn_01.save("dnn_level_01.h5")

# Calculate the time taken and print it out
# print(f'Time taken for training: {time_taken} seconds')


---------------------------------------------------------------------------------
Training DNN
---------------------------------------------------------------------------------
Epoch 1/5
32/82 [==========>...................] - ETA: 0s - loss: 1.4411

82/82 [==============================] - 1s 6ms/step - loss: 0.9644
Epoch 2/5
82/82 [==============================] - 0s 6ms/step - loss: 0.4250
Epoch 3/5
82/82 [==============================] - 0s 6ms/step - loss: 0.3967
Epoch 4/5
82/82 [==============================] - 0s 6ms/step - loss: 0.4926
Epoch 5/5
82/82 [==============================] - 0s 6ms/step - loss: 0.6453


In [ ]:
dnn_01 = load_model("dnn_level_01.h5")


In [ ]:
#DNN
start = time.time()
pred_dnn = dnn_01.predict(X_test_01)
preds_dnn_01 = np.argmax(pred_dnn,axis = 1)
end = time.time()
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)

In [ ]:
# y_test = y_test_01

In [ ]:
#----------------------------------------------------------------
with open(output_file_name, "a") as f: print('Stack model - Strong learner - level 01', file = f)
with open(output_file_name, "a") as f: print('-------------------------------------------------------', file = f)

In [ ]:

print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')
with open(output_file_name, "a") as f: print('DNN', file = f)
pred_label = preds_dnn_01

# pred_label = ypred
#pred_label = label[ypred]

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')

# Acc = ACC(TP_total,TN_total, FP_total, FN_total)
# Precision = PRECISION(TP_total, FP_total)
# Recall = RECALL(TP_total, FN_total)
# F1 = F1(Recall,Precision)
# BACC = BACC(TP_total,TN_total, FP_total, FN_total)
# MCC = MCC(TP_total,TN_total, FP_total, FN_total)


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)

# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1       2      3    4
0  7667.0    59.0   560.0  998.0  0.0
1     3.0  1090.0  5249.0    2.0  0.0
2     1.0   238.0  1489.0    5.0  0.0
3     0.0   348.0   101.0    2.0  0.0
4     0.0     9.0     2.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.5749873758626494
Precision total:  0.36553073628548777
Recall total:  0.3722567111467182
F1 total:  0.30054143561282254
BACC total:  0.3722567111467182
MCC total:  0.4491257396926296


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#SVM
print('---------------------------------------------------------------------------------')
print('Defining SVM Model')
print('---------------------------------------------------------------------------------')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)

#SVM
start = time.time()
clf.fit(X_train_01, y_train_01)
end = time.time()
clf.score(X_train_01, y_train_01)
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
joblib.dump(clf, 'svm_level_01.joblib')


clf = loaded_model = joblib.load('svm_level_01.joblib')


#SVM
start = time.time()
preds_svm_01 = clf.predict(X_test_01)
end = time.time()
time_taken = end - start
with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
print('---------------------------------------------------------------------------------')



---------------------------------------------------------------------------------
Defining SVM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------


In [ ]:
with open(output_file_name, "a") as f: print('-------------------------------------------------------', file = f)
print('---------------------------------------------------------------------------------')
print('CONFUSION MATRIX')
print('---------------------------------------------------------------------------------')
with open(output_file_name, "a") as f: print('SVM', file = f)
pred_label = preds_svm_01

# pred_label = ypred
#pred_label = label[ypred]

confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
all_unique_values = sorted(set(pred_label) | set(y_test_01))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
# confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
# with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
print(confusion_matrix)
with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)
TP_total = sum(TP)
TN_total = sum(TN)
FP_total = sum(FP)
FN_total = sum(FN)

TP_total = np.array(TP_total,dtype=np.float64)
TN_total = np.array(TN_total,dtype=np.float64)
FP_total = np.array(FP_total,dtype=np.float64)
FN_total = np.array(FN_total,dtype=np.float64)



#----------------------------------------------------------------#----------------------------------------------------------------

print('---------------------------------------------------------------------------------')
print('METRICS')
print('---------------------------------------------------------------------------------')

# Acc = ACC(TP_total,TN_total, FP_total, FN_total)
# Precision = PRECISION(TP_total, FP_total)
# Recall = RECALL(TP_total, FN_total)
# F1 = F1(Recall,Precision)
# BACC = BACC(TP_total,TN_total, FP_total, FN_total)
# MCC = MCC(TP_total,TN_total, FP_total, FN_total)


Acc = accuracy_score(y_test_01, pred_label)
Precision = precision_score(y_test_01, pred_label, average='macro')
Recall = recall_score(y_test_01, pred_label, average='macro')
F1 =  f1_score(y_test_01, pred_label, average='macro')
BACC = balanced_accuracy_score(y_test_01, pred_label)
MCC = matthews_corrcoef(y_test_01, pred_label)

# with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1 )
print('BACC total: ', BACC)
print('MCC total: ', MCC)

with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  9042.0   230.0    1.0   11.0  0.0
1   137.0  6207.0    0.0    0.0  0.0
2    10.0  1072.0  650.0    1.0  0.0
3    11.0    21.0    1.0  418.0  0.0
4     1.0     2.0    0.0    8.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9155024406665545
Precision total:  0.7516147186363611
Recall total:  0.6508479677533582
F1 total:  0.6716801573836702
BACC total:  0.6508479677533582
MCC total:  0.8599951010231157


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:

# import sklearn
# from sklearn.model_selection import train_test_split
# split = 0.7

# #AUC ROC
# #---------------------------------------------------------------------

# #AUCROC
# aucroc =[]
# y_array = [y_0,y_1,y_2,y_3,y_4]
# for j in range(0,len(y_array)):
#     # print(j)
#     #------------------------------------------------------------------------------------------------------------
#     X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y_array[j], train_size=split)
    
#     # evaluate the model

#     knn_clf.fit(X_train,y_train)
#     y_pred=knn_clf.predict(X_test) #These are the predicted output value
#     # y_pred = knn_clf.predict_proba(X_test)

    
#     y_scores = y_pred
#     y_true = y_test

#     # model = LGBMClassifier()
#     # model.fit(X_train, y_train)
#     # y_pred = model.predict(X_test)


#     y_scores = y_pred
#     y_true = y_test
    
#     # Calculate AUC-ROC score
#     auc_roc_score= roc_auc_score(y_true, y_scores,  average='weighted')  # Use 'micro' or 'macro' for different averaging strategies
#     # print("AUC-ROC Score class:", auc_roc_score)
#     aucroc.append(auc_roc_score)
#     #-------------------------------------------------------------------------------------------------------    -----
#     # Calculate the average
# average = sum(aucroc) / len(aucroc)

# # Display the result
# # with open(output_file_name, "a") as f:print("AUC ROC Average:", average, file = f)
# print("AUC ROC Average:", average)

# #End AUC ROC